In [ ]:
pip install -r requirements.txt

In [1]:
import json
import re
import pandas as pd
import requests

In [2]:
#query from the curl command of the search request 
query = {"query" : {
  "bool" : {
    "must" : [ {
      "term" : {
        "publicState" : {
          "value" : "RELEASED"
        }
      }
    }, {
      "term" : {
        "versionState" : {
          "value" : "RELEASED"
        }
      }
    }, {
      "bool" : {
        "must" : [ {
          "bool" : {
            "should" : [ {
              "term" : {
                "metadata.creators.person.organizations.identifierPath" : {
                  "value" : "ou_persistent25"
                }
              }
            }, {
              "term" : {
                "metadata.creators.organization.identifierPath" : {
                  "value" : "ou_persistent25"
                }
              }
            } ]
          }
        }, {
          "bool" : {
            "should" : [ {
              "range" : {
                "metadata.datePublishedInPrint" : {
                  "gte" : "2018||/y",
                  "lte" : "2023||/y"
                }
              }
            }, {
              "range" : {
                "metadata.datePublishedOnline" : {
                  "gte" : "2018||/y",
                  "lte" : "2023||/y"
                }
              }
            }, {
              "range" : {
                "metadata.dateAccepted" : {
                  "gte" : "2018||/y",
                  "lte" : "2023||/y"
                }
              }
            }, {
              "range" : {
                "metadata.dateSubmitted" : {
                  "gte" : "2018||/y",
                  "lte" : "2023||/y"
                }
              }
            }, {
              "range" : {
                "metadata.dateModified" : {
                  "gte" : "2018||/y",
                  "lte" : "2023||/y"
                }
              }
            }, {
              "range" : {
                "metadata.dateCreated" : {
                  "gte" : "2018||/y",
                  "lte" : "2023||/y"
                }
              }
            } ]
          }
        } ]
      }
    } ]
  }
},"sort" : [{"metadata.title.keyword":{"order":"asc"}}],"size" : "5000","from" : "0"}

In [3]:
#https://requests.readthedocs.io/en/latest/user/quickstart/#passing-parameters-in-urls
#Sends search request to PuRe REST API with the parameter defined above 

r = requests.post("https://pure.mpg.de/rest/items/search?format=json", json=query)

data = r.json()                                                    #Output is the same as the JSON-Export file

In [4]:
#Create empty DataFrame where Metadata columns will be added
df = pd.DataFrame() 

In [5]:
#Extract PuRe_ID from JSON and add to DataFrame

n = 0
ID = []

for i in data['records']:                                   #Goes through individual record items
    ID.append(data['records'][n]['persistenceId'])
    n += 1
    
df['ID'] = ID

In [6]:
#Extract Creators from JSON and add to DataFrame

n = 0 
creators = []

for i in data['records']:
    c_in = []
    m = 0
    for p in data['records'][n]['data']['metadata']['creators']:           #To capture all persons listed under creators
        if 'person' in data['records'][n]['data']['metadata']['creators'][m]:
            if 'givenName' in data['records'][n]['data']['metadata']['creators'][m]['person']:
                name = ""
                name = data['records'][n]['data']['metadata']['creators'][m]['person']['givenName'] + " " + data['records'][n]['data']['metadata']['creators'][m]['person']['familyName']
                                                                                                                                                    #string of first name + space + last name
                c_in.append(name)
            else:
                name = ""
                name = data['records'][n]['data']['metadata']['creators'][m]['person']['familyName']
                c_in.append(name)
        else:
            name = ""
            name = data['records'][n]['data']['metadata']['creators'][m]['organization']['name']
            c_in.append(name)
        m +=1 
    creators.append("; ".join(c_in))      #combines the names of all creators in this item with the seperator "; " 
    n +=1
    
df['Creators'] = creators

In [7]:
#Extract Title from JSON and add to DataFrame

n = 0
title = []

for i in data['records']:
    title.append(data['records'][n]['data']['metadata']['title'])
    n += 1
    
df['Title'] = title

In [8]:
#Extract Genre from JSON and add to DataFrame

n = 0
genre = []

for i in data['records']:
    genre.append(data['records'][n]['data']['metadata']['genre'])
    n += 1
    
df['Genre'] = genre

In [10]:
#Extract Date from JSON and add to DataFrame

n = 0
date = []
status = []

for i in data['records']:
    if 'datePublishedInPrint' in data['records'][n]['data']['metadata']:                  #Only ever one Date selected by priorities 
        date.append(data['records'][n]['data']['metadata']['datePublishedInPrint'])       #Published before Online before Accepten before Submitted 
        status.append("Published in Print")
    elif 'datePublishedOnline' in data['records'][n]['data']['metadata']:
        date.append(data['records'][n]['data']['metadata']['datePublishedOnline'])
        status.append("Published Online")
    elif 'dateAccepted'in data['records'][n]['data']['metadata']:
        date.append(data['records'][n]['data']['metadata']['dateAccepted'])
        status.append("Accepted")
    elif "dateSubmitted" in data['records'][n]['data']['metadata']:
        date.append(data['records'][n]['data']['metadata']['dateSubmitted'])
        status.append("Submitted")
    elif "dateCreated" in data['records'][n]['data']['metadata']:
        date.append(data['records'][n]['data']['metadata']['dateCreated'])
        status.append("Created")
    else:
        date.append('NA')
        status.append('NA')
    n += 1

df['Status'] = status
df['Date'] = date

In [11]:
#Extract OA Status from JSON and add to DataFrame

n = 0 
OA = []

for i in data['records']:
    if "files" in data['records'][n]['data']:                   #Only if there are files
        oa_in = []
        m = 0
        for p in data['records'][n]['data']['files']:
            if 'oaStatus' in data['records'][n]['data']['files'][m]['metadata']:
                oa_in.append(data['records'][n]['data']['files'][m]['metadata']['oaStatus'])
            else:
                oa_in.append("NA")
            m +=1 
        OA.append("; ".join(oa_in))                           #combines OA status for all files attached separated by "; "
        n +=1
    else:
        OA.append("NA")                                       #Needed so index of existing df and column match 
    
df['OAStatus'] = OA

In [12]:
#Check Dataframe
df

,ID,Creators,Title,Genre,Status,Date,OAStatus
0,item_3499302_2,Michael Franke; Yves Vincent Grossmann,Aktuelle Diskussionen zum Daten- und Softwarem...,TALK_AT_EVENT,Published Online,2023,NA
1,item_2561922_4,Alexander Machado; Katja Heidbach; Johannes Kn...,Analysis of the international journal publishi...,REPORT,Published Online,2018-03-15,NA
2,item_3484757_1,Margit Palzenberger; Johannes Knaus; Jonas Sch...,"BMBF-Verbund-Projekt: ""Interdisziplinarität vo...",REPORT,Published in Print,2021,GOLD
3,item_2595768_2,Inga Overkamp,BrowZine,TALK_AT_EVENT,NA,NA,NA
4,item_2595765_2,Inga Overkamp,Crossref,TALK_AT_EVENT,NA,NA,NA
...,...,...,...,...,...,...,...
92,item_3331716_6,Ralf Schimmer; Ádám Dér; Colleen Campbell; Kai...,The DEAL Cost Modeling Tool,DATA_PUBLICATION,Published Online,2021-07-27,NA
93,item_3348997_1,Stefanie Andergassen,The DOI-Service of the Max Planck Digital Library,TALK_AT_EVENT,Published in Print,2021,NA
94,item_3388027_3,Yves Vincent Grossmann; Michael Franke,Trust Global and Collaborate Local to Changing...,TALK_AT_EVENT,Published Online,2022-06-08,NA
95,item_3508029_2,Yves Vincent Grossmann,Using Labfolder within the Max Planck Society,TALK_AT_EVENT,Published Online,2023,NA


In [13]:
#Write Dataframe as csv into file 

institute = "MPDL"
df.to_csv(institute + '.csv')